## Ejemplo de clasificación multiclase

Se trabajará con el conjunto de datos llamado [Fashion MNIST built-in](https://github.com/zalandoresearch/fashion-mnist). 

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

# Los datos ya están ordenados en entrenamiento y prueba
(train_data, train_labels), (test_data, test_labels) = fashion_mnist.load_data()

In [ ]:
# Visualizar la primera instancia de entrenamiento
print(f"Instancia de entrenamiento:\n{train_data[0]}\n") 
print(f"Etiqueta de la instancia: {train_labels[0]}")

In [ ]:
# Tamaño de los datos
train_data.shape, train_labels.shape, test_data.shape, test_labels.shape

In [ ]:
# Tamaño de una instancia
train_data[0].shape, train_labels[0].shape

Se tiene 60 000 instancias de entrenamiento de tamaño (28, 28) y una etiqueta, así como 10 000 instancias de prueba de tamaño  (28, 28).


In [ ]:
# Gráfico de una instancia
import matplotlib.pyplot as plt

plt.imshow(train_data[7]);

In [ ]:
# Etiqueta de la instancia
train_labels[7]

El nombre de las clases se puede encontrar en el repositorio de Github de [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist#labels)).


In [ ]:
nombres = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
           'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
# Instancia y su etiqueta
plt.imshow(train_data[17], cmap=plt.cm.binary)
plt.title(nombres[train_labels[17]]);

In [ ]:
import random

plt.figure(figsize=(7, 7))

for i in range(4):
  ax = plt.subplot(2, 2, i + 1)
  rand_index = random.choice(range(len(train_data)))
  plt.imshow(train_data[rand_index], cmap=plt.cm.binary)
  plt.title(nombres[train_labels[rand_index]])
  plt.axis(False)

Se construirá un modelo que trate sobre la relación entre los valores de los píxeles y sus etiquetas. 

Dado que es un problema de clasificación multiclase, se necesita realizar algunas modificaciones a la arquitectura:

* **Tamaño de entrada**: se tiene que considerar tensores de 28x28 = 784 (alto y ancho de las imágenes), es decir, un vector de tamaño 784

* **Tamaño de salida**: tendrá que ser 10, dado que se requiere que el modelo prediga 10 clases diferentes.
  * Se modificará la función de activación para que sea de tipo [`"softmax"`](https://www.tensorflow.org/api_docs/python/tf/keras/activations/softmax). ESta función brinda una serie de valores entre 0 & 1 (el mismo tamaño que el tamaño de salida, que aproximadamente suma a 1. El índice con el valor más alto es predicho como la clase más probable.
* Para la función de costo se requiere utilizar la función de pérdida multiclase. 
  * Dado que las etiquetas son enteras, se utilizará [`tf.keras.losses.SparseCategoricalCrossentropy()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy). Si las etiquetas estuviesen en formato one-hot (e.g. algo como `[0, 0, 1, 0, 0...]`), se utilizaría [`tf.keras.losses.CategoricalCrossentropy()`](https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy).
* Se utilizará el parámetro `validation_data` al utilizar la función `fit()`. Esto brindará una idea de cómo se comporta el movimiento en un conjunto de prueba durante el entrenamiento.


In [ ]:
tf.random.set_seed(42)

model_1 = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape = (28, 28)), # Capa de entrada (la capa flatten convierte 28x28 en 784)
  tf.keras.layers.Dense(4, activation = "relu"),
  tf.keras.layers.Dense(4, activation = "relu"),
  tf.keras.layers.Dense(10, activation = "softmax") # Capa de salida de tamaño 10 (softmax)
])

model_1.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(), 
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

non_norm_history = model_1.fit(train_data,
                               train_labels,
                               epochs = 10,
                               validation_data = (test_data, test_labels)) 

In [ ]:
# Verificación del modelo
model_1.summary()

El modelo brinda aproximadamente 35% de exactitud (accuracy) luego de utilizar 10 épocas. Esto es mejor que algo completamente aleatorio donde, dado que se tiene 10 clases, se predeciría cada una con 10% de probabilidad. 

Para mejorar este comportamiento se puede normalizar los datos (llevarlos a un rango entre 0 y 1). 

In [ ]:
# Verificación del mínimo y máximo 
train_data.min(), train_data.max()

In [ ]:
# Dividir los datos de las imágenes entre el máximo valor (255)
train_data = train_data / 255.0
test_data = test_data / 255.0

# Verificar el valor mínimo y máximo actual
train_data.min(), train_data.max()

In [ ]:
tf.random.set_seed(42)

model_2 = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(4, activation="relu"),
  tf.keras.layers.Dense(4, activation="relu"),
  tf.keras.layers.Dense(10, activation="softmax")
])

model_2.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

norm_history = model_2.fit(train_data,
                           train_labels,
                           epochs = 10,
                           validation_data = (test_data, test_labels))

Para visualizar mejor qué está sucediendo, se puede graficar las curvas de pérdida.

In [ ]:
import pandas as pd

# Curvas de pérdida de los datos no normalizados
pd.DataFrame(non_norm_history.history).plot(title="Datos no normalizados")

# Curvas de pérdida de los datos normalizados
pd.DataFrame(norm_history.history).plot(title="Datos normalizados");

A partir de estos gráficos, se observa que el modelo con datos normalizados "aprende" mucho más rápido que el modelo sin normalización.

A continuación se modificará el factor de aprendizaje (learning rate)

In [ ]:
tf.random.set_seed(42)

model_3 = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(4, activation="relu"),
  tf.keras.layers.Dense(4, activation="relu"),
  tf.keras.layers.Dense(10, activation="softmax")
])

model_3.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

# Creación de un "callback" para el factor de aprendizaje
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-3 * 10**(epoch/20))

# Entrenamiento del modelo
find_lr_history = model_3.fit(train_data,
                              train_labels,
                              epochs = 40,       # tal vez no se requiera 100 épocas
                              validation_data=(test_data, test_labels),
                              callbacks=[lr_scheduler])

In [ ]:
# Gráfico del factor de aprendizaje (learning rate)
import numpy as np

lrs = 1e-3 * (10**(np.arange(40)/20))
plt.semilogx(lrs, find_lr_history.history["loss"]) # eje x-axis en escala logarítmica
plt.xlabel("Learning rate")
plt.ylabel("Loss")
plt.title("Finding the ideal learning rate");

Según el gráfico parece que el valor óptimo podría estar alrededor de 0.001. Se reentrenará el modelo utilizando este factor de aprendizaje

In [ ]:
tf.random.set_seed(42)

model_4 = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(4, activation="relu"),
  tf.keras.layers.Dense(4, activation="relu"),
  tf.keras.layers.Dense(10, activation="softmax")
])

model_4.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), # Valor "ideal"
                metrics = ["accuracy"])

history = model_4.fit(train_data,
                      train_labels,
                      epochs = 20,
                      validation_data = (test_data, test_labels))

Luego de tener un modelo con un factor de aprendizaje adecuado y con un comportamiento relativamente adecuado, se puede realizar alguna de las siguientes alternativas.

* Evaluar su rendimiento utilizando otras métricas de clasificación (como la  [matriz de confusión](https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py) o un [reporte de clasificación](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)).
* Evaluar sus predicciones (a través de visualizaciónes).
* Mejorar la exactitud del modelo (entrenándo por más tiempo o modificando la arquitectura).
* Guardar el modelo y exportarlo para uso en una aplicación.

Primero se utilizará una matriz de confusión para visualizar las predicciones de las diferentes clases.

In [ ]:
import itertools
from sklearn.metrics import confusion_matrix

def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15): 
  """Genera una matriz de confusión comparando las predicciones y las etiquetas reales

  Si se pasa classes, se etiquetará la matriz de confusión. De lo contrario, se utilizará
  valores enteros.

  Args:
    y_true: Arreglo de etiquetas reales (igual tamaño que y_pred).
    y_pred: Arreglo de etiquetas predichas (igual tamaño que y_true).
    classes: Arreglo de etiquetas de clase (ejm. en formato de string). Si es `None`, se usa etiquetas enteras
    figsize: Tamaño de la salida de la figura (default=(10, 10)).
    text_size: Tamaño de la salida del texto  (default=15).
  
  Returns:
    Una matriz de confusión

  """  
  # Creación de la matriz de confusión
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0]

  # Gráfico de la figura
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) 
  fig.colorbar(cax)

  # Hay una lista de clases?
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])
  
  # Etiqueta de los ejes
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes),
         yticks=np.arange(n_classes), 
         xticklabels=labels,
         yticklabels=labels)
  
  # Etiquetas del eje x en la parte inferior
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  # Umbral para los colores
  threshold = (cm.max() + cm.min()) / 2.

  # Gráfico de texto en cada celda
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
             horizontalalignment="center",
             color="white" if cm[i, j] > threshold else "black",
             size=text_size)

In [ ]:
# Predicciones con el modelo más reciente
y_probs = model_4.predict(test_data)

# Primeras 5 predicciones
y_probs[:5]

La salida es un vector de probabilidades. Para encontrar el valor más alto (la clase más probable) se puede utilizar [`argmax()`](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html).

In [ ]:
# Clase predicha y su etiqueta, para la primera instancia
y_probs[0].argmax(), nombres[y_probs[0].argmax()]

In [ ]:
# Convertir todas las predicciones, de probabilidades a etiquetas
y_preds = y_probs.argmax(axis=1)

# Visualización de las primeras 10 predicciones
y_preds[:10]

In [ ]:
# Matriz de confusión de Scikit learn
from sklearn.metrics import confusion_matrix

confusion_matrix(y_true = test_labels, 
                 y_pred = y_preds)

In [ ]:
# Matriz de confusión gráfica
make_confusion_matrix(y_true = test_labels, 
                      y_pred = y_preds,
                      classes = nombres,
                      figsize = (15, 15),
                      text_size = 10)

Parece que el modelo se confunde entre `Shirt` y `T-shirt/top`. Para analizar y tratar de comprender un poco más este problema se puede visualizar algunos ejemplos.

In [ ]:
import random

# Función que grafica una imagen aleatoria junto con su predicción
def plot_random_image(model, images, true_labels, classes):
  # Imagen aleatoria
  i = random.randint(0, len(images))
  
  # Predicciones y etiquetas
  target_image = images[i]
  pred_probs = model.predict(target_image.reshape(1, 28, 28))
  pred_label = classes[pred_probs.argmax()]
  true_label = classes[true_labels[i]]

  # Gráfico de la imagen
  plt.imshow(target_image, cmap=plt.cm.binary)

  # Cambiar el color de los títulos según si la predicción es correcta o no
  if pred_label == true_label:
    color = "green"
  else:
    color = "red"
  plt.xlabel("Pred: {} {:2.0f}% (True: {})".format(pred_label,
                                                   100*tf.reduce_max(pred_probs),
                                                   true_label),
             color=color)

In [ ]:
# Visualizar una imagen aleatoria y su predicción
plot_random_image(model = model_4, 
                  images = test_data, 
                  true_labels = test_labels, 
                  classes = nombres)

### ¿Qué patrones aprende el modelo?

Se obtendrá una lista de las capas en el modelo más reciente (`model_14`) usando el atributo `layers`.

In [ ]:
# Capas del modelo más reciente
model_4.layers

In [ ]:
# Extracción de una capa particular
model_4.layers[1]

Se puede encontrar los parámetros aprendidos por cada capa usando`get_weights()`, que retorna los pesos (weights) y sesgos (biases). Cada neurona tiene un vector de sesgo (bias), el cual se encuentra ligado a una matriz de pesos. Los valores de sesgo se inicializan a cero por defecto (pero pueden tener otras inicializaciones) y determinan cuánto los patrones de los pesos correspondientes deben influir en la siguiente capa.


In [ ]:
# Obtener los patrones de una capa en la red
weights, biases = model_4.layers[1].get_weights()

weights, weights.shape

In [ ]:
biases, biases.shape

In [ ]:
# Resumen del modelo
model_4.summary()

In [ ]:
from tensorflow.keras.utils import plot_model

# Resumen de las entradas y salidas de cada capa
plot_model(model_4, show_shapes=True)